In [1]:
# default setting
import argparse
from batchgenerators.utilities.file_and_folder_operations import *
from unetr_pp.run.default_configuration import get_default_configuration
from unetr_pp.paths import default_plans_identifier
from unetr_pp.utilities.task_name_id_conversion import convert_id_to_task_name
import numpy as np
import torch

seed = 42
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

C:\Users\user\anaconda3\envs\ailab\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


unetr_pp_raw_data_base is not defined and model can only be used on data for which preprocessed files are already present on your system. model cannot be used for experiment planning and preprocessing like this. If this is not intended, please read run_training_synapse.sh/run_training_acdc.sh for information on how to set this up properly.
unetr_pp_preprocessed is not defined and model can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
RESULTS_FOLDER is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read run_training_synapse.sh/run_training_acdc.sh for information on how to set this up.


In [2]:
# to improve the efficiency set the last two true
def main():
    
    # import parser
    # In sh file, python ../unetr_pp/run/run_training.py 3d_fullres unetr_pp_trainer_synapse 2 0
    parser = argparse.ArgumentParser()
    #parser.add_argument("network", type=str, default='3d_fullres')
    #parser.add_argument("network_trainer", type=str, default='unetr_pp_trainer_synapse')
    #parser.add_argument("task", type=int, help="can be task name or task id", default=2)
    #parser.add_argument("fold", type=int, help='0, 1, ..., 5 or \'all\'', default=0)
    parser.add_argument("-val", "--validation_only", help="use this if you want to only run the validation",
                        action="store_true")
    parser.add_argument("-c", "--continue_training", help="use this if you want to continue a training",
                        action="store_true")
    parser.add_argument("-p", help="plans identifier. Only change this if you created a custom experiment planner",
                        default=default_plans_identifier, required=False)
    parser.add_argument("--use_compressed_data", default=False, action="store_true",
                        help="If you set use_compressed_data, the training cases will not be decompressed. "
                             "Reading compressed data is much more CPU and RAM intensive and should only be used if "
                             "you know what you are doing", required=False)
    parser.add_argument("--deterministic",
                        help="Makes training deterministic, but reduces training speed substantially. I (Fabian) think "
                             "this is not necessary. Deterministic training will make you overfit to some random seed. "
                             "Don't use that.",
                        required=False, default=False, action="store_true")
    parser.add_argument("--npz", required=False, default=False, action="store_true", help="if set then model will "
                                                                                          "export npz files of "
                                                                                          "predicted segmentations "
                                                                                          "in the validation as well. "
                                                                                          "This is needed to run the "
                                                                                          "ensembling (if needed)")
    parser.add_argument("--find_lr", required=False, default=False, action="store_true",
                        help="not used here, just for fun")
    parser.add_argument("--valbest", required=False, default=False, action="store_true",
                        help="hands off. This is not intended to be used")
    parser.add_argument("--fp32", required=False, default=True, action="store_true",
                        help="disable mixed precision training and run old school fp32")
    parser.add_argument("--val_folder", required=False, default="validation_raw",
                        help="name of the validation folder. No need to use this for most people")
    parser.add_argument("--disable_saving", required=False, action='store_true',
                        help="If set nnU-Net will not save any parameter files (except a temporary checkpoint that "
                             "will be removed at the end of the training). Useful for development when you are "
                             "only interested in the results and want to save some disk space")
    parser.add_argument("--disable_postprocessing_on_folds", required=False, action='store_true',
                        help="Running postprocessing on each fold only makes sense when developing with nnU-Net and "
                             "closely observing the model performance on specific configurations. You do not need it "
                             "when applying nnU-Net because the postprocessing for this will be determined only once "
                            "Usually running postprocessing on each fold is computationally cheap, but some users have "
                             "reported issues with very large images. If your images are large (>600x600x600 voxels) "
                             "you should consider setting this flag.")
    parser.add_argument('--val_disable_overwrite', action='store_false', default=True,
                        help='Validation does not overwrite existing segmentations')
    parser.add_argument('--disable_next_stage_pred', action='store_true', default=False,
                        help='do not predict next stage')
    parser.add_argument('-pretrained_weights', type=str, required=False, default=None,
                        help='path to nnU-Net checkpoint file to be used as pretrained model (use .model '
                             'file, for example model_final_checkpoint.model). Will only be used when actually training. '
                             'Optional. Beta. Use with caution.')

    args = parser.parse_args('')
    
    # set variables
    task = '2'
    fold = '0'
    network = '3d_fullres'
    network_trainer = 'unetr_pp_trainer_synapse'
    
    validation_only = args.validation_only
    plans_identifier = args.p # unetr_pp_Plansv2.1
    find_lr = args.find_lr
    disable_postprocessing_on_folds = args.disable_postprocessing_on_folds

    use_compressed_data = args.use_compressed_data
    decompress_data = not use_compressed_data

    deterministic = args.deterministic
    valbest = args.valbest

    fp32 = args.fp32
    run_mixed_precision = not fp32

    val_folder = args.val_folder
    
    # convert_id_to_task_name 확인하기
    #### id_name: Task002_Synapse
    '''
    if not task.startswith("Task"):
        task_id = int(task)
        task = convert_id_to_task_name(task_id)
    '''
    if int(task) == 2
        task = 'Task002_Synapse'
    else:
        raise Exception("It is not Synapse file. You must use task=2.")
    
    # fold는 그대로
    if fold == 'all':
        pass
    else:
        fold = int(fold)
    
    # Setting 1
    plans_file, output_folder_name, dataset_directory, batch_dice, stage, \
    trainer_class = get_default_configuration(network, task, network_trainer, plans_identifier)

    if trainer_class is None:
        raise RuntimeError("Could not find trainer class in unetr_pp.training.network_training")
    '''
    I am using stage 1 from these plans
    I am using batch dice + CE loss

    I am using data from this folder:  
    ../DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse\Task002_Synapse\unetr_pp_Data_plans_v2.1
    '''
    
    '''
    My trainer class is:  <class 'unetr_pp.training.network_training.unetr_pp_trainer_synapse.unetr_pp_trainer_synapse'>
    For that I will be using the following configuration:
    num_classes:  13 -> 9(8 segmentation + background)
    '''
    
    # stage = 1
    '''
    {'batch_size': 2, 'num_pool_per_axis': [3, 5, 5], 'patch_size': array([ 64, 128, 128]), 
    'median_patient_size_in_voxels': array([148, 512, 512], dtype=int64), 
    'current_spacing': array([3.        , 0.75781244, 0.75781244]), 
    'original_spacing': array([3.        , 0.75781244, 0.75781244]), 
    'do_dummy_2D_data_aug': True, 
    'pool_op_kernel_sizes': [[2, 2, 2], [2, 2, 2], [2, 2, 2]], 
    'conv_kernel_sizes': [[1, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]]}
    '''
    #print(output_folder_name)
    output_folder_name += 'fold_'+ str(fold)
    os.makedirs(output_folder_name, exist_ok=True)
    trainer = trainer_class(plans_file, fold, output_folder=output_folder_name, dataset_directory=dataset_directory,
                            batch_dice=batch_dice, stage=stage, unpack_data=decompress_data,
                            deterministic=deterministic, fp16=run_mixed_precision)
    if args.disable_saving:
        trainer.save_final_checkpoint = False  # whether or not to save the final checkpoint
        trainer.save_best_checkpoint = False  # whether or not to save the best checkpoint according to
        # self.best_val_eval_criterion_MA
        trainer.save_intermediate_checkpoints = True  # whether or not to save checkpoint_latest. We need that in case
        # the training chashes
        trainer.save_latest_only = True  # if false it will not store/overwrite _latest but separate files each
    
    
    # C:\Users\user\Desktop\Liver\2023\unetr_plus_plus-main\DATASET_Synapse\unetr_pp_raw\unetr_pp_raw_data\Task02_Synapse\Task002_Synapse\unetr_pp_Data_plans_v2.1_stage1
    ### 위의 폴더에 preprocessed된 데이터 넣기. (일단 원본 데이터 ㄱㄱ)
    trainer.initialize(not validation_only)

    if find_lr:
        trainer.find_lr()
    else:
        if not validation_only:
            if args.continue_training:
                # -c was set, continue a previous training and ignore pretrained weights
                trainer.load_latest_checkpoint()
            else:
                # new training without pretraine weights, do nothing
                pass

            trainer.run_training()
        else:
            if valbest:
                trainer.load_best_checkpoint(train=False)
            else:
                trainer.load_final_checkpoint(train=False)

        trainer.network.eval()

        # predict validation
        trainer.validate(save_softmax=args.npz, validation_folder_name=val_folder,
                         run_postprocessing_on_folds=not disable_postprocessing_on_folds,
                         overwrite=args.val_disable_overwrite)

In [3]:
if __name__ == "__main__":
    main()

RuntimeError: Could not find a task with the ID 2. Make sure the requested task ID exists and that nnU-Net knows where raw and preprocessed data are located (see Documentation - Installation). Here are your currently defined folders:
unetr_pp_preprocessed=None
RESULTS_FOLDER=None
unetr_pp_raw_data_base=None
If something is not right, adapt your environemnt variables.